# Advent of Code - 2024 - Day 9 - Problem 2

https://adventofcode.com/2024/day/9

## Load Source Data

Load the map data into `DATA`.

In [1]:
f = open("data/day9.txt", "r")
DATA = f.read()
f.close()

# Create DiskEntry Class

In [2]:
class DiskEntry:

    def __init__(self, file_id, position, size):
        self.file_id = file_id
        self.position = position
        self.size = size

    def __str__(self):
        return f"({self.file_id},{self.position},{self.size})"

# Create Disk Class

In [3]:
class Disk:

    def __init__(self):
        self.files = list()
        self.gaps = list()

    def parse_map(self, map):
        entry_is_file = True
        file_id = -1
        position = 0
        for m in DATA:
            size = int(m)

            if entry_is_file:
                file_id += 1
                file = DiskEntry(file_id, position, size)
                self.files.append(file)
            else:
                if (size > 0):
                    gap = DiskEntry(-1, position, size)
                    self.gaps.append(gap)

            position += size
            entry_is_file = not entry_is_file

    def find_gap(self, size, max_position):
        for diskEntry in self.gaps:
            if diskEntry.position > max_position: return None
            if diskEntry.size >= size: return diskEntry
        return None
    
    def get_gap_at(self, position):
        for diskEntry in self.gaps:
            if diskEntry.position == position: return diskEntry
        return None
    
    def consolidate_gaps(self):

        # Sort the gaps by position
        #
        self.gaps.sort(key = lambda de: de.position)
        gap_map = {gap.position: gap for gap in self.gaps}

        idx = 0
        while idx < len(self.gaps):
            gap = self.gaps[idx]
            next_position = gap.position + gap.size
            if next_position in gap_map.keys():
                next_gap = gap_map[next_position]
                gap.size += next_gap.size
                self.gaps.remove(next_gap)
            else:
                idx += 1

    def move_file(self, file, gap):

        # Swap the file and gap positions.
        #
        position = gap.position
        gap.position = file.position
        file.position = position

        # Create a new gap if file is smaller than target gap.
        #
        if file.size < gap.size:
            self.gaps.append(DiskEntry(-1, file.position + file.size, gap.size - file.size))

        self.consolidate_gaps()

    def compact_files(self):

        # Sort the files by descending position
        #
        self.files.sort(key = lambda de: -de.position)

        for file in self.files:
            gap = self.find_gap(file.size, file.position)
            if gap != None:
                self.move_file(file, gap)

    def compute_checksum(self):
        checksum = 0
        for file in self.files:
            for idx in range(file.size):
                checksum += file.file_id * (file.position + idx)
        return checksum          

## Process the Data

In [4]:
disk = Disk()
disk.parse_map(DATA)

disk.compact_files()

checksum = disk.compute_checksum();

print(f"checksum = {checksum}")

checksum = 6326952672104
